# Isolation Forest

![basic isolation](img\basic-isolation-example.png)

FICOU MUITO GRANDE, DEPOIS EU EDITO

## 1. Introdução

Isolation Forest é um algoritmo eficiente de deteção de dados extremos, especialmente em datasets com alta dimensionalidade.
O algoritmo constroi uma random forest em cada árvore de decisão cresce aleatoriamente. Em cada nó ele seleciona uma feature aleatoriamente, depois seleciona um threshold aleatório para dividir o dataset em dois. O algoritmo divide o dataset gradualmente até que todas as instâncias estejam isoladas das outras instâncias. Outliers normalmente estão longe de outras instâncias, logo esses datapoints tendem a ser isolados em menos etapas que datapoints "normais". 

## 2. Dataset

In [ ]:
A GENTE PODE USAR UM DATASET QUALQUER OU CRIAR UM DATASET 